In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2019-07-07 16:37:06--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   159MB/s    in 2.5s    

2019-07-07 16:37:09 (159 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [0]:
!unzip ./uncased_L-12_H-768_A-12.zip

Archive:  ./uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
import json

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import csv
import os
import modeling
import optimization
import tokenization
import tensorflow as tf

W0707 16:38:02.181702 140377027782528 deprecation_wrapper.py:119] From /content/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None,text_c=None,text_d=None,text_e=None,text_f=None,label=None):

    
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.text_c = text_c
    self.text_d = text_d
    self.text_e = text_e
    self.text_f = text_f
    self.label = label

In [0]:
class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

In [0]:
class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
examples=[]
with open("gdrive/My Drive/ARC-V1-Feb2018/ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Train.jsonl") as f:
      for line in f.readlines():
         example = json.loads(line)
         examples.append(example)
        

In [80]:
examples[1]

{'answerKey': 'B',
 'id': 'MCAS_2009_5_6516',
 'question': {'choices': [{'label': 'A',
    'text': 'The refrigerator door is smooth.'},
   {'label': 'B', 'text': 'The refrigerator door contains iron.'},
   {'label': 'C', 'text': 'The refrigerator door is a good conductor.'},
   {'label': 'D', 'text': 'The refrigerator door has electric wires in it.'}],
  'stem': 'Which of the following statements best explains why magnets usually stick to a refrigerator door?'}}

In [0]:
def process_input(json_inputs):
  input_examples=[]
  for input_ in json_inputs:
    text_a="".join(input_['question']['stem'])
    entities=[]
    for i in range(0,5):
      try:
        entities.append(input_['question']['choices'][i]['text'])
      except IndexError:
        entities.append("no option")
        
    label=input_['answerKey'] 
    if(label =='1'):
      label='A'
    if(label == '2'):
      label = 'B'
    if(label=='3'):
      label = 'C'
    if(label=='4'):
      label ='D'
   
    e1="".join(entities[0])
    e2="".join(entities[1])
    e3 = "".join(entities[2])
    e4 = "".join(entities[3])
    e5 = "".join(entities[4])
    
    input_examples.append(InputExample(input_["id"],text_a,e1,e2,e3,e4,e5,label))
  return input_examples 
input_examples=process_input(examples)

In [0]:
examples_cv=[]
with open("gdrive/My Drive/ARC-V1-Feb2018/ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Dev.jsonl") as f:
      for line in f.readlines():
         example = json.loads(line)
         examples_cv.append(example)
        
input_examples_cv=process_input(examples_cv)

In [0]:
set_label = []
for i in range(22):
  set_label.append(input_examples[i].label)

In [85]:
set(set_label)

{'A', 'B', 'C', 'D'}

In [86]:
print(input_examples[0].text_a)
print(input_examples[0].text_b)
print(input_examples[0].text_c)
print(input_examples[0].text_d)
print(input_examples[0].text_e)
print(input_examples[0].text_f)
print(input_examples[0].label)

George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?
dry palms
wet palms
palms covered with oil
palms covered with lotion
no option
A


In [0]:
def _truncate_seq_pair(tokens_a, tokens_b, tokens_c,tokens_d,tokens_e,tokens_f,max_length):
  """Truncates a sequence pair in place to the maximum length."""


  while True:
    total_length = len(tokens_a) + len(tokens_b)+len(tokens_c) + len(tokens_d) + len(tokens_e) + len(tokens_f)
    if total_length <= max_length:
      break
    
    tokens_a.pop()


In [0]:
def get_labels():
  return ['A',\
 'B',\
 'C',\
 'D',\
 'E'         
 ]

In [0]:
def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""

  if isinstance(example, PaddingInputExample):
    return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        is_real_example=False)

  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i

  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)
    #print(tokens_b)
  tokens_c = ["none"]
  if example.text_c:
    tokens_c = tokenizer.tokenize(example.text_c)
    #print(tokens_c)
  tokens_d  =["none"]
  if example.text_d:
    tokens_d  = tokenizer.tokenize(example.text_d)
    
  tokens_e  =["none"]
  if example.text_e:
    tokens_e  = tokenizer.tokenize(example.text_e) 
    
  tokens_f = ["none"] 
  if example.text_f:
    tokens_f = tokenizer.tokenize(example.text_f)
    
  if tokens_b:

    _truncate_seq_pair(tokens_a, tokens_b,tokens_c,tokens_d,tokens_e,tokens_f,max_seq_length - 7)
  

  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(-1)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(-1)
  tokens.append("[SEP]")
  segment_ids.append(-1)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)
  if tokens_c:
    for token in tokens_c:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)
  if tokens_d:
    for token in tokens_d:
      tokens.append(token)
      segment_ids.append(2)
    tokens.append("[SEP]")
    segment_ids.append(2)  
  if tokens_e:
    for token in tokens_e:
      tokens.append(token)
      segment_ids.append(3)
    tokens.append("[SEP]")
    segment_ids.append(3)  
  if tokens_f:
    for token in tokens_f:
      tokens.append(token)
      segment_ids.append(4)
    tokens.append("[SEP]")
    segment_ids.append(4)
    
  input_ids = tokenizer.convert_tokens_to_ids(tokens)
  #print(len(input_ids))
  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)
  
  #if len(input_ids)>128:
    #print(tokens_a,tokens_b,tokens_c)
  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]
  if ex_index < 5:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("label: %s (id = %d)" % (example.label, label_id))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id,
      is_real_example=True)
  return feature

In [0]:
nput_examples[5].text_a,input_examples[5].text_b,input_examples[5].text_c,input_examples[5].text_d,input_examples[5].text_e,input_examples[5].label

("Tunisia ' s flag carrier Tunisair scored 60 million Tunisian dinars ( 41 . 15 million US dollars ) in net income in 2009 despite world economic downturn which badly hit air transport industry the company said on Friday .",
 'Tunisair',
 'Tunisia',
 'org:country_of_headquarters')

In [90]:
print(input_examples[0].text_a)
print(input_examples[0].text_b)
print(input_examples[0].text_c)
print(input_examples[0].text_d)
print(input_examples[0].text_e)
print(input_examples[0].text_f)
print(input_examples[689].label)

George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?
dry palms
wet palms
palms covered with oil
palms covered with lotion
no option
D


In [0]:
x = convert_single_example(3, input_examples[3], get_labels(), 128,
                           tokenizer)

128


In [0]:
print(x.input_ids)
print(x.input_mask)
print(x.segment_ids)
print(x.label_id)

[101, 2029, 1997, 1996, 2206, 2003, 2019, 2742, 1997, 2019, 6509, 3512, 5080, 1029, 102, 3967, 10014, 102, 9055, 102, 4542, 16531, 102, 4157, 8962, 102, 2053, 5724, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [0]:
def file_based_convert_examples_to_features(
    examples, label_list, max_seq_length, tokenizer, output_file):
  """Convert a set of `InputExample`s to a TFRecord file."""

  writer = tf.python_io.TFRecordWriter(output_file)

  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    features["label_ids"] = create_int_feature([feature.label_id])
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())
  writer.close()

In [0]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

In [0]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  output_layer = model.get_pooled_output()

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_real_example = None
    if "is_real_example" in features:
      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
    else:
      is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
#     for var in tvars:
#       init_string = ""
#       if var.name in initialized_variable_names:
#         init_string = ", *INIT_FROM_CKPT*"
#       tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
#                       init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(
            labels=label_ids, predictions=predictions, weights=is_real_example)
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn,
                      [per_example_loss, label_ids, logits, is_real_example])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities},
          scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)


    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [0]:
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    features.append(feature)
  return features

In [0]:
do_lower_case=True
init_checkpoint="./uncased_L-12_H-768_A-12/bert_model.ckpt"
tokenization.validate_case_matches_checkpoint(do_lower_case,
                                                init_checkpoint)

In [0]:
bert_config_file="./uncased_L-12_H-768_A-12/bert_config.json"
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

In [0]:
output_dir="output_dir"
tf.gfile.MakeDirs(output_dir)


label_list = get_labels()

In [0]:
vocab_file="./uncased_L-12_H-768_A-12/vocab.txt"
tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=True)

In [0]:
tpu_cluster_resolver = None
master=None
iterations_per_loop=1000
save_checkpoints_steps=1000
num_tpu_cores=8
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      master=master,
      model_dir=output_dir,
      save_checkpoints_steps=save_checkpoints_steps,
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=iterations_per_loop,
          num_shards=num_tpu_cores,
          per_host_input_for_training=is_per_host))

In [0]:
do_train=True
num_train_epochs=9
warmup_proportion=0.1
train_batch_size=24
train_examples = None
num_train_steps = None
num_warmup_steps = None

if do_train:
    train_examples = input_examples
    num_train_steps = int(
        len(train_examples) / train_batch_size * num_train_epochs)
    num_warmup_steps = int(num_train_steps * warmup_proportion)

In [0]:
learning_rate=1e-5
use_tpu=False
model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=init_checkpoint,
      learning_rate=learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=use_tpu,
      use_one_hot_embeddings=use_tpu)

In [103]:
eval_batch_size=train_batch_size=predict_batch_size=16
estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=train_batch_size,
      eval_batch_size=eval_batch_size,
      predict_batch_size=predict_batch_size)

W0707 17:53:07.036693 140377027782528 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7fabc8e2cea0>) includes params argument, but params are not passed to Estimator.
W0707 17:53:07.040364 140377027782528 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.


<h2>Model training</h2>

In [104]:
max_seq_length=170
train_file = os.path.join(output_dir, "train.tf_record")
file_based_convert_examples_to_features(
        train_examples, label_list, max_seq_length, tokenizer, train_file)
print("***** Running training *****")
print("  Num examples = %d", len(train_examples))
print("  Batch size = %d", train_batch_size)
print("  Num steps = %d", num_train_steps)
train_input_fn = file_based_input_fn_builder(
        input_file=train_file,
        seq_length=max_seq_length,
        is_training=True,
        drop_remainder=True)

170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170


In [105]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

In [0]:
mytest_file = os.path.join(output_dir, "mytest.tf_record")
file_based_convert_examples_to_features(exa, label_list,
                                            max_seq_length, tokenizer,
                                            mytest_file)

<h2>Evaluating the model</h2>

In [0]:
eval_examples = input_examples_cv
num_actual_eval_examples = len(eval_examples)

In [114]:
eval_file = os.path.join(output_dir, "eval.tf_record")
file_based_convert_examples_to_features(
        eval_examples, label_list, max_seq_length, tokenizer, eval_file)

tf.logging.info("***** Running evaluation *****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                    len(eval_examples), num_actual_eval_examples,
                    len(eval_examples) - num_actual_eval_examples)
tf.logging.info("  Batch size = %d", eval_batch_size)

170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170
170


In [0]:
eval_input_fn = file_based_input_fn_builder(
        input_file=eval_file,
        seq_length=max_seq_length,
        is_training=False,
        drop_remainder=False)

In [0]:
result = estimator.evaluate(input_fn=eval_input_fn, steps=None)

In [117]:
output_eval_file = os.path.join(output_dir, "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
      tf.logging.info("***** Eval results *****")
      for key in sorted(result.keys()):
        print("  %s = %s", key, str(result[key]))
        print("%s = %s\n" % (key, str(result[key])))

  %s = %s eval_accuracy 0.21404682
eval_accuracy = 0.21404682

  %s = %s eval_loss 1.5966659
eval_loss = 1.5966659

  %s = %s global_step 844
global_step = 844

  %s = %s loss 1.5950656
loss = 1.5950656



<h2>Predictions</h2>

In [0]:
examples_test=[]
with open("gdrive/My Drive/ARC-V1-Feb2018/ARC-V1-Feb2018-2/ARC-Challenge/ARC-Challenge-Test.jsonl") as f:
      for line in f.readlines():
         example = json.loads(line)
         examples_test.append(example)
        
input_examples_test=process_input(examples_test)

In [77]:
input_examples_test[150].label

'C'

In [0]:
def get_predictions(text_a,text_b,text_c,text_d,text_e,text_f,label):
  exa=[]
  text_a=text_a
  text_b=text_b
  text_c =text_c
  text_d = text_d
  text_e = text_e
  text_f=text_f
  label = label
  exa.append(
            InputExample(guid=1000, text_a=text_a, text_b=text_b,text_c=text_c,text_d=text_d,text_e=text_e,text_f=text_f,label=label))
  mytest_file = os.path.join(output_dir, "mytest.tf_record")
  file_based_convert_examples_to_features(exa, label_list,
                                              max_seq_length, tokenizer,
                                              mytest_file)
  predict_drop_remainder=False
  mytest_input_fn = file_based_input_fn_builder(
          input_file=mytest_file,
          seq_length=max_seq_length,
          is_training=False,
          drop_remainder=predict_drop_remainder)
  result = estimator.predict(input_fn=mytest_input_fn)
  for (i, prediction) in enumerate(result):
          #tf.logging.info(i)
          print("************************probability*********************************",prediction["probabilities"])

In [78]:
import numpy as np
outputs = []
for i in range(10):
  
  text_a=input_examples_test[i].text_a

  text_b=input_examples_test[i].text_b

  text_c = input_examples_test[i].text_c

  text_d = input_examples_test[i].text_d

  text_e = input_examples_test[i].text_e

  text_f = input_examples_test[i].text_f

  label = input_examples_test[i].label
  
  predictions_prob = get_predictions(text_a,text_b,text_c,text_d,text_e,text_f,label)
  np.argmax(predictions_prob)
  np.max(predictions_prob)
  outputs.append(np.argmax(predictions_prob))
  print(label)

170
************************probability********************************* [0.26003313 0.31086117 0.14544459 0.27889854 0.00476258]
C
170
************************probability********************************* [0.19258508 0.10781594 0.12901746 0.56655335 0.00402817]
B
170
************************probability********************************* [0.28862333 0.3127576  0.32073018 0.07141327 0.00647562]
C
170
************************probability********************************* [0.11551436 0.09069177 0.06005534 0.721886   0.01185256]
D
170
************************probability********************************* [0.14188863 0.46147552 0.16906597 0.2192921  0.00827784]
D
170
************************probability********************************* [0.15847743 0.44914412 0.27993557 0.09721778 0.01522514]
B
170
************************probability********************************* [0.21924129 0.3274577  0.29183823 0.15523866 0.00622406]
C
170
************************probability********************************* [0.

In [74]:
outputs

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]